In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import string
import nltk
import pickle
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
ps = PorterStemmer() # stemming
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('arabic')
warnings.filterwarnings('ignore')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Path to your labeled validation dataset
file_path = "/content/drive/MyDrive/labeled_validation_dataset.jsonl"

# Load and preprocess the dataset
df = pd.read_json(file_path, lines=True)
df.drop('example_id', axis=1, inplace=True)

In [4]:
df.head()

,paragraph,summary
0,وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...,يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...
1,ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...,دبلوماسيو الدولتين لم يعترفوا بالعريضة التي قا...
2,قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...,أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب...
3,دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...,مصر هي أم البلاد، وقائدة العرب؛ فهي أرض بلاد ا...
4,السوريون يصرون على استقلال بلادهم : و مثلما رف...,الشعب السوري يصر على استقلال بلدهم من السيطرة ...


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00


In [6]:
!pip install tokenizers

In [7]:
!pip install -U transformers

In [8]:
!pip install sentencepiece
import sentencepiece as spm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00


In [9]:
pip install python-bidi

In [10]:
!pip install torch

In [11]:
!pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186458 sha256=4cd3263bc56e13f400eff2c376cb9d060e300125b5cf467d7b3f0c364288b0d4
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

model_name="abdalrahmanshahrour/arabartsummarization"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline("text2text-generation",model=model,tokenizer=tokenizer)

In [13]:
import re
import regex

def preprocess_text(text):
    # Remove non-Arabic characters except periods and commas
    text = regex.sub(r"[^\p{Script=Arabic}\s.,]", "", text)

    # Remove diacritics
    text = regex.sub(r"\p{M}", "", text)

    # Remove tatweel (elongation character)
    text = text.replace("ـ", "")

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text)

    # Remove leading/trailing spaces
    text = text.strip()
    print(text)
    return text




In [14]:
import nltk

nltk.download('stopwords')
import nltk
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:

stop_words = set(nltk.corpus.stopwords.words('arabic'))

def remove_stopwords(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text, language='arabic')

    # Remove stopwords from the tokenized text
    filtered_tokens = [token for token in tokens if token not in string.punctuation and token not in stop_words]

    # Join the filtered tokens back into a single string
    filtered_text = ' '.join(filtered_tokens)

    print(filtered_text)
    return filtered_text


In [16]:
def summarize_text(text):
    # معالجة مسبقة للنص
    preprocessed_text = preprocess_text(text)

   # Tokenize the input text
    inputs = tokenizer([preprocessed_text], return_tensors="pt", padding=True, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

In [17]:
def get_summary(text):
    text = preprocessor.preprocess(text)

    max_input_length = tokenizer.model_max_length - tokenizer.num_special_tokens_to_add(pair=True)
    segments = [text[i:i + max_input_length] for i in range(0, len(text), max_input_length)]

    generated_texts = []
    for segment in segments:
        result = pipeline(
            segment,
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            repetition_penalty=3.0,
            max_length=200,
            length_penalty=1.0,
            no_repeat_ngram_size=3
        )[0]['generated_text']
        generated_texts.append(result)

    result = " ".join(generated_texts)
    return result

In [60]:
# List to store summaries and references
summaries = []
references = []

for sen in tqdm(df["paragraph"][:3]):
    summary_text = get_summary(sen)
    summaries.append(summary_text)

for reference in tqdm(df["summary"]):
    references.append(reference)

100%|██████████| 154/154 [00:00<00:00, 677067.94it/s]


In [36]:
summaries[2]

'في عام 1920 قامت فرنسا بتقسيم سوريا إلى ثلاث ولايات هي حلب ودمشق واللاذقية وذلك بهدف تأديب السوريين على تصديهم لفرنسا.'

In [35]:
references[2]

'أعلن غورو الانتداب الفرنسي على سوريا لكي يعاقب السوريين لأنهم تصدوا لفرنسا، وكان غورو يريد زيادة التنافس التجاري بين حلب ودمشق، وأعلن دمشق عاصمة لكي يغضب الحلبيين وساند الامتيازات الاقتصادية في حلب لكي يغضب الدمشقيين، وكان غورو يريد حلب ترفض أن تتعاون مع دمشق، لكن فرنسا لم تستطع تطبيق فكرة الولايات المنفصلة في سوريا، وفرنسا قسمت حلب ودمشق في ولاية وبعدت ولاية اللاذقية عنهم، وقام سلطان باشا الأطرش بقيادة ثورة سورية كبيرة، وقام إبراهيم هنانو بقيادة ثورة وقاموا بإحباط المشروع الفرنسي.'

In [25]:
!pip install rouge

In [61]:
from rouge import Rouge
def calculate_rouge(reference, summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    return scores[0]

In [62]:
# حساب مقاييس ROUGE
rouge_scores = []

for reference, summary in zip(references, summaries):
    rouge_score = calculate_rouge(reference, summary)
    rouge_scores.append(rouge_score)

In [63]:
rouge_scores[2]

{'rouge-1': {'r': 0.203125, 'p': 0.3023255813953488, 'f': 0.24299064939820078},
 'rouge-2': {'r': 0.05263157894736842,
  'p': 0.08695652173913043,
  'f': 0.0655737657941417},
 'rouge-l': {'r': 0.140625, 'p': 0.20930232558139536, 'f': 0.1682242942580139}}